## Demonstração Graph Neural Networks

# Primeira GCN

In [ ]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')
print(dataset.x[0])
print(dataset.edge_index)


### Define o modelo - usando uma GCN (Graph Convolution Network)
Este modelo visa prever a classe de um determinado nó

Veja as implementações de outras redes em pytorch: https://pytorch-geometric.readthedocs.io/en/latest/cheatsheet/gnn_cheatsheet.html


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
                             #entrada                 , saida
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

### Instância do modelo e treino

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001,  weight_decay=5e-4)

#definindo o modo de treino
model.train()

for epoch in range(1000):
    optimizer.zero_grad()
    out = model(data)
    
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    print(loss.item())
    

### Teste

In [ ]:
#definindo o modo de teste
model.eval()

pred = model(data).argmax(dim=1)

correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())

print('Accuracy: {:.4f}'.format(acc))

# Link prediction - com GAE (Graph AutoEncoder)

In [24]:
from torch import nn
import torch_geometric.transforms as T
import torch.optim as optim
from torch_geometric.datasets import Planetoid
import torch 

from torch.nn import ReLU

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]
print(data)
data.train_mask = data.val_mask = data.test_mask = None

transform = T.RandomLinkSplit(is_undirected=True,add_negative_train_samples=True,split_labels=True)

train_data, val_data, test_data = transform(data)
train_data = train_data.to(device)
val_data=val_data.to(device)
test_data = test_data.to(device)

print()
print(test_data.pos_edge_label)
print()
print(train_data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0')

Data(x=[2708, 1433], edge_index=[2, 7392], y=[2708], pos_edge_label=[3696], pos_edge_label_index=[2, 3696], neg_edge_label=[3696], neg_edge_label_index=[2, 3696])


### Define o encoder

In [ ]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels) 
        self.conv2 = GCNConv(2 * out_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)
    

### Define o autoencoder

In [ ]:
from torch_geometric.nn import GAE

# parâmetros
out_channels = 64
num_features = dataset.num_features

epochs = 100

# modelo - Graph Auto-Encoder (GAE)
model = GAE(GCNEncoder(num_features, out_channels))
model = model.to(device)

print(model)

x = train_data.x.to(device)

train_pos_edge_index = train_data.pos_edge_label_index.to(device)

# inicialização o optimizador
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### Treino e teste

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
           
           #model.test(...) vem da classe GAE
    return model.test(z, pos_edge_index, neg_edge_index)



In [ ]:
for epoch in range(1, epochs + 1):
    loss = train()

    auc, ap = test(val_data.pos_edge_label_index, val_data.neg_edge_label_index)
    print(f'Epoch: {epoch:03d}, AUC: {auc:.4f}, AP: {ap:.4f}')
    

In [ ]:
#Teste
auc, ap = test(test_data.pos_edge_label_index, test_data.neg_edge_label_index)
print('AUC: {:.4f}, AP: {:.4f}'.format( auc, ap))

In [ ]:
#Decodificando

temp = model.decode(test_data.x, test_data.pos_edge_label_index)

threshold = torch.tensor([0.5]).to(device)

#Atribui 0 ou 1 de acordo com o threshold
results = (temp>threshold).float()

train_acc = torch.sum(test_data.pos_edge_label == results)

final_train_acc = train_acc/test_data.pos_edge_label.shape[0]
final_train_acc


### Ferramenta interessante para analisar diversos experimentos: TensorBoard
